In [ ]:
import numpy as np
import pandas as pd
import string
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_table('../input/train.tsv', engine='c')
df_test = pd.read_table('../input/test.tsv', engine='c')

In [ ]:
df1 = df.drop('train_id',axis=1)
df_test_p = df_test.drop('test_id',axis=1)

In [ ]:
df_staticstics_by_brand = df1.groupby(['brand_name'])["price"].describe()[['min','max','mean']]

In [ ]:
staticstics_for_nan_brand = df1[df1['brand_name'].isnull()]['price'].describe()[['min','max','mean']]

In [ ]:
def statictics_price_for_brand(row, df_staticstics_by_brand, staticstics_for_nan_brand):
    brand_name = row['brand_name']
    if brand_name != brand_name:
        return staticstics_for_nan_brand
    if brand_name in df_staticstics_by_brand.index:
        return df_staticstics_by_brand.loc[brand_name]
    return staticstics_for_nan_brand

In [ ]:
df1[['brand_min_price', 'brand_max_price', 'brand_mean_price']] = df1.apply(lambda row: statictics_price_for_brand(row, df_staticstics_by_brand, staticstics_for_nan_brand), axis=1)
df_test_p[['brand_min_price', 'brand_max_price', 'brand_mean_price']] = df_test_p.apply(lambda row: statictics_price_for_brand(row, df_staticstics_by_brand, staticstics_for_nan_brand), axis=1)

In [ ]:
df2 = df1

In [ ]:
def category_to_list(row):
    category_name = row['category_name']
    categories = []
    if category_name == category_name:
        categories = category_name.split("/")
    missing_categories = 4 - categories.__len__()
    if missing_categories < 0:
        return categories[:4]
    for i in np.arange(1,missing_categories,1):
        categories = categories + ["None"]
    return categories

In [ ]:
categories_df =  pd.DataFrame()
categories_df['categories_list'] = df2.apply(lambda row: category_to_list(row), axis=1)
categories_df = categories_df.replace(r'\s+', "None", regex=True)
categories_df = categories_df.fillna("None")

In [ ]:
categories_test_df =  pd.DataFrame()
categories_test_df['categories_list'] = df_test_p.apply(lambda row: category_to_list(row), axis=1)
categories_test_df = categories_test_df.replace(r'\s+', "None", regex=True)
categories_test_df = categories_test_df.fillna("None")

In [ ]:
df2[['cat1', 'cat2', 'cat3','cat4']] = pd.DataFrame(categories_df.categories_list.values.tolist(), index= categories_df.index)
df_test_p[['cat1', 'cat2', 'cat3','cat4']] = pd.DataFrame(categories_test_df.categories_list.values.tolist(), index= categories_test_df.index)

In [ ]:
df_statictics_by_cat3 = df2.groupby(['cat1','cat2','cat3','cat4'])["price"].describe()[['min','max','mean']]

In [ ]:
statictics_for_nan_cat = df2[df2['category_name'].isnull()]['price'].describe()[['min','max','mean']]

In [ ]:
df2 = pd.merge(df2, df_statictics_by_cat3, how='left', left_on=['cat1','cat2','cat3','cat4'], right_index=True)
df_test_p = pd.merge(df_test_p, df_statictics_by_cat3, how='left', left_on=['cat1','cat2','cat3','cat4'], right_index=True)
df2[['min','max','mean']] = df2[['min','max','mean']].fillna(statictics_for_nan_cat)
df_test_p[['min','max','mean']] = df_test_p[['min','max','mean']].fillna(statictics_for_nan_cat)


In [ ]:
df2 = df2.rename(columns={'min': 'min_cat_price', 'max': 'max_cat_price', 'mean': 'mean_cat_price'})
df_test_p = df_test_p.rename(columns={'min': 'min_cat_price', 'max': 'max_cat_price', 'mean': 'mean_cat_price'})
df2.head(1)

In [ ]:
df2['brand_name'] = df2['brand_name'].fillna("None")
df_test_p['brand_name'] = df_test_p['brand_name'].fillna("None")
df_statictics_by_cat_and_brand = df2.groupby(['brand_name','cat1','cat2','cat3'])["price"].describe()[['min','max','mean']]


In [ ]:
df_statictics_by_cat_and_brand.head(1)

In [ ]:
df2 = pd.merge(df2, df_statictics_by_cat_and_brand, how='left', left_on=['brand_name','cat1','cat2','cat3'], right_index=True)
df_test_p = pd.merge(df_test_p, df_statictics_by_cat_and_brand, how='left', left_on=['brand_name','cat1','cat2','cat3'], right_index=True)


In [ ]:
df2[['min','max','mean']] = df2[['min','max','mean']].fillna(statictics_for_nan_cat)
df_test_p[['min','max','mean']] = df_test_p[['min','max','mean']].fillna(statictics_for_nan_cat)

In [ ]:
df3 = df2.drop('category_name',axis=1)
df3 = df3.drop('cat1',axis=1)
df3 = df3.drop('cat2',axis=1)
df3 = df3.drop('cat3',axis=1)
df3 = df3.drop('cat4',axis=1)

In [ ]:
df_test_p = df_test_p.drop('category_name',axis=1)
df_test_p = df_test_p.drop('cat1',axis=1)
df_test_p = df_test_p.drop('cat2',axis=1)
df_test_p = df_test_p.drop('cat3',axis=1)

In [ ]:
import re

df3['name_len'] = df3['name'].str.len()
df3['desc_len'] = df3['item_description'].str.len()
df_test_p['name_len'] = df_test_p['name'].str.len()
df_test_p['desc_len'] = df_test_p['item_description'].str.len()

df3['description_numbers_count'] = df3['item_description'].str.count('[0-9]')
df_test_p['description_numbers_count'] = df_test_p['item_description'].str.count('[0-9]')


In [ ]:
df3['contains_new'] = df3['item_description'].str.count('new', flags=re.IGNORECASE) + df3['name'].str.count('new', flags=re.IGNORECASE)
df_test_p['contains_new'] = df_test_p['item_description'].str.count('new', flags=re.IGNORECASE) + df_test_p['name'].str.count('new', flags=re.IGNORECASE)
df3['contains_sold'] = df3['item_description'].str.count('sale|sold', flags=re.IGNORECASE) + df3['name'].str.count('sale|sold', flags=re.IGNORECASE)
df_test_p['contains_sold'] = df_test_p['item_description'].str.count('sale|sold', flags=re.IGNORECASE) + df_test_p['name'].str.count('sale|sold', flags=re.IGNORECASE)
df3['contains_gift'] = df3['item_description'].str.count('sale|sold', flags=re.IGNORECASE) + df3['name'].str.count('gift|free', flags=re.IGNORECASE)
df_test_p['contains_gift'] = df_test_p['item_description'].str.count('sale|sold', flags=re.IGNORECASE) + df_test_p['name'].str.count('gift|free', flags=re.IGNORECASE)


In [ ]:
df4 = df3.drop('name',axis=1)
df4 = df4.drop('item_description',axis=1)
df4 = df4.drop('brand_name',axis=1)
df_test_p = df_test_p.drop('brand_name',axis=1)
df_test_p = df_test_p.drop('name',axis=1)
df_test_p = df_test_p.drop('item_description',axis=1)
df4 = df4.drop('price',axis=1)

In [ ]:
df4 = df4.fillna(0)
df_test_p = df_test_p.fillna(0)

In [ ]:
from scipy import sparse
#X_train = sparse.hstack((transformed_name,transformed_item_description,df4.iloc[:, :].values)).A
X_train = df4.iloc[:, :].values
y_train = df['price'].iloc[:].values
#X_test = sparse.hstack((transformed_name_test,transformed_item_description_test,df_test_p.iloc[:, :].values)).A
X_test = df_test_p.iloc[:, :].values

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
X_test

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
df_out = pd.DataFrame()
df_out['test_id'] = df_test.test_id
df_out['price'] = y_pred
df_out['price'] = df_out['price'].clip(0)
df_out.to_csv("sample_submission.csv", index=False)